#Multi-Agent Order Management System with MongoDB

This notebook implements a multi-agent system for managing product orders, inventory, and deliveries using:

smolagents for agent management
MongoDB for data persistence
DeepSeek Chat as the LLM model

##Setting Up MongoDB Atlas

Create a free MongoDB Atlas account at https://www.mongodb.com/cloud/atlas/register

- Create a new cluster (free tier is sufficient)
- Configure network access by adding your IP address
- Create a database user with read/write permissions
- Get your connection string from Atlas UI (Click "Connect" > "Connect your application")
- Replace <password> in the connection string with your database user's password
- Enable network access from your IP address in the Network Access settings

###Security Considerations

When working with MongoDB Atlas:

- Never commit connection strings with credentials to version control
- Use environment variables or secure secret management
- Restrict database user permissions to only what's needed
- Enable IP allowlist in Atlas Network Access settings

##Setup

First, let's install required dependencies:

In [ ]:
!pip install smolagents pymongo litellm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 32.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 30.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
!pip install 'smolagents[litellm]'

In [ ]:
from smolagents.agents import ToolCallingAgent
from smolagents import tool, LiteLLMModel, ManagedAgent, CodeAgent
from pymongo import MongoClient
from datetime import datetime
from typing import List, Dict, Optional
import os

# Initialize LLM model
MODEL_ID = "gemini/gemini-2.0-flash"
MONGODB_URI = os.getenv('MONGO_URI')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

In [ ]:
# MongoDB Connection manager
mongoclient = MongoClient(MONGODB_URI, appname="devrel.showcase.multi-smolagents")
db = mongoclient.warehouse

NameError: name 'MongoClient' is not defined

In [ ]:
@tool
def check_stock(product_id: str) -> Dict:
    """Query product stock level.

    Args:
        product_id: Product identifier

    Returns:
        Dict containing product details and quantity
    """
    return db.products.find_one({"_id": product_id})

@tool
def update_stock(product_id: str, quantity: int) -> bool:
    """Update product stock quantity.

    Args:
        product_id: Product identifier
        quantity: Amount to decrease from stock

    Returns:
        bool: Success status
    """
    result = db.products.update_one(
        {"_id": product_id},
        {"$inc": {"quantity": -quantity}}
    )
    return result.modified_count > 0

In [ ]:
@tool
def create_order( products: any, address: str) -> str:
    """Create new order for all provided products.

    Args:
        products: List of products with quantities
        address: Delivery address

    Returns:
        str: Order ID message
    """
    order = {
        "products": products,
        "status": "pending",
        "delivery_address": address,
        "created_at": datetime.now()
    }
    result = db.orders.insert_one(order)
    return f"Successfully ordered : {str(result.inserted_id)}"

In [ ]:
from bson.objectid import ObjectId
@tool
def update_delivery_status(order_id: str, status: str) -> bool:
    """Update order delivery status to in_transit once a pending order is provided

    Args:
        order_id: Order identifier
        status: New delivery status is being set to in_transit or delivered

    Returns:
        bool: Success status
    """
    if status not in ["pending", "in_transit", "delivered", "cancelled"]:
        raise ValueError("Invalid delivery status")

    result = db.orders.update_one(
        {"_id":  ObjectId(order_id), "status": "pending"},
        {"$set": {"status": status}}
    )
    return result.modified_count > 0

In [ ]:
class OrderManagementSystem:
    """Multi-agent order management system"""
    def __init__(self, model_id: str = MODEL_ID):
        self.model = LiteLLMModel(model_id=model_id, api_key=GOOGLE_API_KEY)



        # Create agents
        self.inventory_agent = ToolCallingAgent(
            tools=[check_stock, update_stock],
            model=self.model,
            max_iterations=10
        )

        self.order_agent = ToolCallingAgent(
            tools=[create_order],
            model=self.model,
            max_iterations=10
        )

        self.delivery_agent = ToolCallingAgent(
            tools=[update_delivery_status],
            model=self.model,
            max_iterations=10
        )

        # Create managed agents
        self.managed_agents = [
            ManagedAgent(self.inventory_agent, "inventory", "Manages product inventory"),
            ManagedAgent(self.order_agent, "orders", "Handles order creation"),
            ManagedAgent(self.delivery_agent, "delivery", "Manages delivery status")
        ]

        # Create manager agent
        self.manager = CodeAgent(
            tools=[],
            system_prompt="""For each order:
            1. Create the order document
            2. Update the inventory
            3. Set deliviery status to in_transit

            Use relevant agents:  {{managed_agents_descriptions}}  and you can use {{authorized_imports}}
            """,
            model=self.model,
            managed_agents=self.managed_agents,
            additional_authorized_imports=["time", "json"]
        )

    def process_order(self, orders: List[Dict]) -> str:
        """Process a set of orders.

        Args:
            orders: List of orders each has address and products

        Returns:
            str: Processing result
        """
        return self.manager.run(
            f"Process the following  {orders} as well as substract the ordered items from inventory."
            f"to be delivered to relevant addresses"
        )

In [ ]:
def add_sample_products():
    db.products.delete_many({})
    sample_products = [
        {"_id": "prod1", "name": "Laptop", "price": 999.99, "quantity": 10},
        {"_id": "prod2", "name": "Smartphone", "price": 599.99, "quantity": 15},
        {"_id": "prod3", "name": "Headphones", "price": 99.99, "quantity": 30}
    ]

    db.products.insert_many(sample_products)
    print("Sample products added successfully!")

# Uncomment to add sample products
# add_sample_products()

In [ ]:
# Initialize system
system = OrderManagementSystem()

# Create test orders
test_orders = [
    {
        "products": [
            {"product_id": "prod1", "quantity": 2},
            {"product_id": "prod2", "quantity": 1}
        ],
        "address": "123 Main St"
    },
    {
        "products": [
            {"product_id": "prod3", "quantity": 3}
        ],
        "address": "456 Elm St"
    }
]

# Process order
result = system.process_order(
    orders=test_orders
)

print("Orders processing result:", result)